In [1]:
import numpy as np
from wfdb import readsignal

In [2]:
# Testing rdsamp format 212
sig, fields=readsignal.rdsamp('sampledata/100', sampto=9999)
print(sig)
print(len(sig))

[[-0.145 -0.065]
 [-0.145 -0.065]
 [-0.145 -0.065]
 ..., 
 [ 0.515  0.41 ]
 [ 0.71   0.32 ]
 [ 0.795  0.045]]
9999


In [3]:
# Testing rdsamp format 16
sig, fields=readsignal.rdsamp('sampledata/test01_00s')
print(sig)

[[ 0.1  -0.08 -0.57 -0.66]
 [ 0.11 -0.06 -0.56 -0.66]
 [ 0.13 -0.06 -0.55 -0.67]
 ..., 
 [-0.27 -0.17  0.13  0.15]
 [-0.27 -0.18  0.13  0.15]
 [-0.26 -0.18  0.12  0.16]]


In [3]:
# Testing rdsamp format 80
sig, fields=readsignal.rdsamp('sampledata/3000003_0003')
print(sig)

[[-5  0]
 [-5  0]
 [-6  0]
 ..., 
 [-7  6]
 [-8  6]
 [-7  6]]


In [3]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/a103l')
print(sig)
print(fields)
#gains=np.array([1,2,3])
#sig=np.subtract(sig, gains)
#print(sig)

[[-0.02359597  0.86758555  0.48220271]
 [-0.03698082  0.98298479  0.5443735 ]
 [-0.06292259  0.85979087  0.47821229]
 ..., 
 [-0.04084449  0.7493346   0.5150838 ]
 [-0.04719194  0.7581749   0.50957702]
 [-0.04677798  0.7615019   0.5028731 ]]
{'initvalue': [-171, 9127, 6042], 'baseline': [0, 0, 0], 'nsamp': 82500, 'gain': [7247.0, 10520.0, 12530.0], 'nsampseg': [], 'info': ['Asystole', 'False alarm'], 'byteoffset': ['24', '24', '24'], 'fmt': ['16', '16', '16'], 'basetime': '', 'nsig': 3, 'basedate': '', 'units': ['mV', 'mV', 'NU'], 'fs': 250.0, 'filename': ['a103l.mat', 'a103l.mat', 'a103l.mat'], 'signame': ['II', 'V', 'PLETH'], 'nseg': 1}


In [5]:
np.sctypes

{'complex': [numpy.complex64, numpy.complex128, numpy.complex256],
 'float': [numpy.float16, numpy.float32, numpy.float64, numpy.float128],
 'int': [numpy.int8, numpy.int16, numpy.int32, numpy.int64],
 'others': [bool, object, str, str, numpy.void],
 'uint': [numpy.uint8, numpy.uint16, numpy.uint32, numpy.uint64]}

In [6]:
a=np.array(range(0,12,4))
a=np.hstack((a,a))
print(a)

listofnps=[a,a]
print(listofnps)


listofnps.append("hello")
print(listofnps)



[0 4 8 0 4 8]
[array([0, 4, 8, 0, 4, 8]), array([0, 4, 8, 0, 4, 8])]
[array([0, 4, 8, 0, 4, 8]), array([0, 4, 8, 0, 4, 8]), 'hello']


In [19]:
def givethree():
    return 1, 2, 3

list1=[]
list1.append(givethree()[1])
print(list1)

[2]


In [5]:
numbs=['1','2','3']

num=[float(i) for i in numbs]
print(num)

[1.0, 2.0, 3.0]
